<a href="https://colab.research.google.com/github/eshal26/Bank-Costumer-Churn-Prediction/blob/main/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>